# 01 — Clean & Merge FPDS with BTS and OpenSanctions

In [ ]:
# Install missing packages into the ACTIVE kernel if needed
import sys, subprocess, importlib
def ensure(pkg, pip_name=None):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name or pkg])
for mod in ["pandas", "numpy"]:
    ensure(mod)
try:
    importlib.import_module("rapidfuzz")
except Exception:
    try:
        ensure("rapidfuzz")
    except Exception:
        ensure("fuzzywuzzy"); ensure("python-Levenshtein","python-Levenshtein")
print("[INFO] Dependencies are ready.")

In [ ]:
import pandas as pd, numpy as np, re
from pathlib import Path
from src.utils import find_project_root, ensure_dir, simple_vendor_normalize, clean_naics, to_usps_state

# Prefer RapidFuzz; fallback to fuzzywuzzy
try:
    from rapidfuzz import process, fuzz
except Exception:
    from fuzzywuzzy import process, fuzz

ROOT = find_project_root(Path.cwd())
raw = ROOT / "data" / "raw"
proc = ROOT / "data" / "processed"
ensure_dir(proc)

print("ROOT:", ROOT)

In [ ]:
# ---- Load FPDS ----
fpds_path = raw / "fpds_contracts.csv"
if not fpds_path.exists():
    raise FileNotFoundError(f"Expected {fpds_path}")

fpds = pd.read_csv(fpds_path)
# normalize column names
fpds.columns = fpds.columns.str.strip().str.lower().str.replace(" ","_")
# parse dates + add temporal parts
fpds["action_date"] = pd.to_datetime(fpds.get("action_date"), errors="coerce")
fpds["year_month"] = fpds["action_date"].dt.to_period("M").astype(str)
# currency/numeric
if "dollar_obligated" in fpds.columns:
    fpds["dollar_obligated"] = (
        fpds["dollar_obligated"].astype(str).str.replace(",", "", regex=False).str.strip().replace({"": np.nan})
    )
    fpds["dollar_obligated"] = pd.to_numeric(fpds["dollar_obligated"], errors="coerce")
# vendor & NAICS features
fpds["vendor_group"] = fpds.get("vendor_name","").apply(simple_vendor_normalize)
fpds["naics_code_clean"] = fpds.get("naics_code","").apply(clean_naics)
fpds["naics_sector2"] = fpds["naics_code_clean"].str[:2]
# location normalization
fpds["place_of_performance_state"] = fpds.get("place_of_performance_state","").astype(str).str.upper().str.strip()
fpds["state_usps"] = fpds["place_of_performance_state"].apply(to_usps_state)

fpds.head()

In [ ]:
# ---- Load BTS Port Performance ----
bts_path = raw / "bts_port_performance.csv"
if not bts_path.exists():
    raise FileNotFoundError(f"Expected {bts_path}")

bts = pd.read_csv(bts_path)
bts.columns = bts.columns.str.strip().str.lower()
# ensure string types that match FPDS year_month and USPS codes
bts["year_month"] = bts["year_month"].astype(str)
bts["state"] = bts["state"].astype(str).str.upper().str.strip()
bts = bts[["year_month","state","average_turnaround_time","percent_on_time"]]
bts.head()

In [ ]:
# ---- Merge FPDS + BTS on (state, year_month) ----
merged = fpds.merge(
    bts,
    how="left",
    left_on=["state_usps","year_month"],
    right_on=["state","year_month"],
    suffixes=("","_bts")
)
merged.drop(columns=["state"], inplace=True, errors="ignore")
print(merged[["vendor_group","state_usps","year_month","average_turnaround_time","percent_on_time"]].head(6))

In [ ]:
# ---- Load OpenSanctions and flag vendors via fuzzy match ----
san_path = raw / "opensanctions_vendors.csv"
if not san_path.exists():
    raise FileNotFoundError(f"Expected {san_path}")

san = pd.read_csv(san_path)
san.columns = san.columns.str.strip().str.lower()
san["entity_norm"] = san["entity_name"].apply(simple_vendor_normalize)

# Build a set/list of unique normalized sanctioned entities
sanction_list = san["entity_norm"].dropna().unique().tolist()

def fuzzy_flag(name: str, candidates, threshold=95):
    if not isinstance(name, str) or not name:
        return (0, "")
    # use RapidFuzz if available (above import order ensures one exists)
    match = process.extractOne(name, candidates, scorer=fuzz.WRatio)
    if match is None:
        return (0, "")
    cand, score, _ = match if isinstance(match, tuple) and len(match) == 3 else (match[0], match[1], None)
    return (int(score), cand if score >= threshold else "")

merged["sanction_score"], merged["sanction_match"] = zip(*merged["vendor_group"].map(lambda x: fuzzy_flag(x, sanction_list)))
merged["sanctioned_vendor"] = merged["sanction_score"].ge(95)

merged.head(8)

In [ ]:
# ---- Save processed/enriched dataset ----
out = proc / "fpds_cleaned.csv"  # keep same filename for downstream notebooks
merged.to_csv(out, index=False)
print(f"[OK] wrote {out} with shape {merged.shape}")